In [2]:
import cv2

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D
from sklearn.preprocessing import OrdinalEncoder
#from keras.utils import to_categorical
from keras.preprocessing import image
from keras.optimizer_v2 import adam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline

In [4]:
train = pd.read_csv('dataset_500.csv', encoding='ISO-8859-1')    # reading the csv file
train.head()      # printing first five rows of the file

,Image name,Ophthalmologic department,Retinopathy grade,Risk of macular edema
0,20051019_38557_0100_PP.tif,Service Ophtalmologie LariboisiÃ¨re,3,1
1,20051020_43808_0100_PP.tif,Service Ophtalmologie LariboisiÃ¨re,0,0
2,20051020_43832_0100_PP.tif,Service Ophtalmologie LariboisiÃ¨re,1,0
3,20051020_43882_0100_PP.tif,Service Ophtalmologie LariboisiÃ¨re,2,0
4,20051020_43906_0100_PP.tif,Service Ophtalmologie LariboisiÃ¨re,3,2


In [5]:
train = train[['Image name', 'Retinopathy grade']]
train.head()

,Image name,Retinopathy grade
0,20051019_38557_0100_PP.tif,3
1,20051020_43808_0100_PP.tif,0
2,20051020_43832_0100_PP.tif,1
3,20051020_43882_0100_PP.tif,2
4,20051020_43906_0100_PP.tif,3


In [6]:
print(train['Retinopathy grade'].unique())

[3 0 1 2]


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
from google.colab import drive
drive.mount('OdinSet')

Mounted at OdinSet


In [9]:
def get_histogram(image, bins):
    # array with size of bins, set to zeros
    histogram = np.zeros(bins)
    
    # loop through pixels and sum up counts of pixels
    for pixel in image:
        histogram[pixel] += 1
    
    # return our final result
    return histogram

In [10]:
def cumsum(a):
    a = iter(a)
    b = [next(a)]
    for i in a:
        b.append(b[-1] + i)
    return np.array(b)

In [22]:
from google.colab.patches import cv2_imshow
scale_percent = 60
train_image = []
for i in tqdm(range(train.shape[0])):
    image = cv2.imread('/content/OdinSet/MyDrive/OdinSet/'+train['Image name'][i])
    

   
    dim = (1382, 921)
    resized = cv2.resize(image, (1382, 921))

    RGBimg = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(RGBimg, cv2.COLOR_RGB2GRAY)
    flat = gray.flatten()
    hist = get_histogram(flat, 256)
    cs = cumsum(hist)
    nj = (cs - cs.min()) * 255
    N = cs.max() - cs.min()
    # re-normalize the cumsum
    cs = nj / N
    # cast it back to uint8 since we can't use floating point values in images
    cs = cs.astype('uint8')
    img_new = cs[flat]
    img_new = np.reshape(img_new, gray.shape)
    train_image.append(img_new)

X = np.array(train_image)

  0%|          | 1/500 [00:01<14:50,  1.79s/it]


error: ignored

In [13]:
plt.imshow(X[2])

NameError: ignored

In [ ]:
X.shape

(500, 921, 1382)

In [ ]:
y = np.array(train.drop(['Image name'],axis=1))
y.shape

(500, 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [ ]:
print(X_train.shape)
print(X_test.shape)


(400, 921, 1382)
(100, 921, 1382)
